# Import dependencies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

2025-07-10 17:17:59.585296: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-10 17:18:00.421043: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-10 17:18:01.306465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752147182.430173    5224 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752147182.658974    5224 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752147184.119468    5224 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

# Classes and num_classes

In [3]:
import os

In [4]:

train_dir = 'archive/train'
# List all directories inside train_dir
classes = [name for name in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, name))]

print(classes)

['Target_Spot', 'Tomato_mosaic_virus', 'healthy', 'Bacterial_spot', 'Leaf_Mold', 'Early_blight', 'Tomato_Yellow_Leaf_Curl_Virus', 'Late_blight', 'Septoria_leaf_spot', 'Spider_mites Two-spotted_spider_mite', 'powdery_mildew']


In [5]:
num_classes = len(classes)

In [6]:
print(num_classes)

11


# Data gen

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'archive/train'
test_dir = 'archive/valid'

# Data generators with basic augmentation for train, only rescale for test
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # for convnext-tiny
    batch_size=32,
    class_mode='categorical'  # for 11 classes
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 25851 images belonging to 11 classes.
Found 6683 images belonging to 11 classes.


# Model definition

In [8]:
convnextt = tf.keras.applications.ConvNeXtTiny(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
    pooling="avg"
)
convnextt.trainable = False  # Freeze base for transfer learning

E0000 00:00:1752147194.780781    5224 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1752147194.835330    5224 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


111650432/111650432 ━━━━━━━━━━━━━━━━━━━━ 96s 1us/step


In [9]:
from tensorflow.keras import layers, models

num_classes = num_classes

model = models.Sequential([
    convnextt,
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])

In [10]:
num_classes

11

# Training

## Top level training

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

early_stop = EarlyStopping(
    monitor='val_accuracy',       
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20,                     
    callbacks=[early_stop]
)

/home/mandip/mandip/projects/ml_projects/6.convnext_finetune/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ImportError: This requires the scipy module. You can install it via `pip install scipy`

## Finetuning

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

convnextt.trainable = True  # Unfreeze the ConvNeXt base

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Lower LR for fine-tuning
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_finetune = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=5,
    callbacks = [early_stop]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 852s 1s/step - accuracy: 0.4382 - loss: 2.7050 - val_accuracy: 0.5815 - val_loss: 1.4154
Epoch 2/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 785s 971ms/step - accuracy: 0.6633 - loss: 1.1516 - val_accuracy: 0.5327 - val_loss: 1.7066
Epoch 3/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 784s 970ms/step - accuracy: 0.7399 - loss: 0.8966 - val_accuracy: 0.6003 - val_loss: 1.5194
Epoch 4/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 783s 970ms/step - accuracy: 0.7718 - loss: 0.7896 - val_accuracy: 0.4437 - val_loss: 2.4721
Epoch 5/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 830s 1s/step - accuracy: 0.8000 - loss: 0.7074 - val_accuracy: 0.4498 - val_loss: 2.9528
Epoch 6/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 826s 1s/step - accuracy: 0.8146 - loss: 0.6794 - val_accuracy: 0.4651 - val_loss: 2.3580


# Prediction

In [7]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.2f}')

209/209 ━━━━━━━━━━━━━━━━━━━━ 40s 192ms/step - accuracy: 0.6046 - loss: 1.5160
Test accuracy: 0.60


# Saving model

In [8]:
model.save("convnext_model.keras")

# Single image prediction

In [9]:
import numpy as np
from tensorflow.keras.preprocessing import image
import os

In [10]:
print(class_names)

NameError: name 'class_names' is not defined

In [ ]:
def predict_image(path):
    # Load and preprocess the image
    img_path = path  # Replace with your image path
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to match input_shape
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize if you trained with normalized images
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])  # Gets the class with highest probability
    train_dir = "/kaggle/input/tomato/train"
    class_names = sorted(os.listdir(train_dir))      
    predicted_class = class_names[predicted_class_index]
    print("Predicted Class:", predicted_class)



In [ ]:
predict_image("/kaggle/input/tomato/train/Early_blight/00e7c4b2-3005-4558-9cfa-235e356cb7a8___RS_Erly.B 7844.JPG")

In [ ]:
predict_image("/kaggle/input/tomato/valid/powdery_mildew/pm100_change_90.jpg")

In [ ]:
predict_image('/kaggle/input/pest-file/Pest2377.jpg')